### Przedmiot: Python 2
### Temat: "Programowanie sieciowe"
### Imię nazwisko: Eduard Povierin
### Grupa: 2TD12A

---

## Cel:
Celem ćwiczenia jest zapoznanie z zasadami budowania aplikacji sieciowych w języku Python, z wykorzystaniem HTML i REST

### Zadanie 1:
Zaprezentować generowanie własnej strony internetowej wyświetlającej tabele z danymi. Strona ma być wygenerowana z wykorzystaniem biblioteki Jinja 2. Dane maja być pobierane z plików CSV

Przykładowy plik CSV:
```
Character,Age
Emil, 1459
Kaine, 18
Nier, 19
```

Plik 'template.html':
```html
<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8" />
      <meta http-equiv="X-UA-Compatible" content="IE=edge" />
      <meta name="viewport" content="width=device-width, initial-scale=1.0" />
      <title>CSV Data Table</title>
   </head>
   <body>
      <table border="1">
         <thead>
            <tr>
               <th>Character</th>
               <th>Age</th>
            </tr>
         </thead>
         <tbody>
            {% for row in data %}
            <tr>
               <td>{{ row.Character }}</td>
               <td>{{ row.Age }}</td>
            </tr>
            {% endfor %}
         </tbody>
      </table>
   </body>
</html>
```

I sam kod programu:

In [ ]:
import csv
import os
from jinja2 import Template

path = os.path.dirname(os.path.abspath(__file__))

with open(f'{path}/data.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)
    
jinja_template = open(f'{path}/template.html', 'r').read()
html_template = Template(jinja_template).render(data=data)

with open(f'{path}/index.html', 'w') as file:
    file.write(html_template)

Po uruchomieniu programu, strona powinna wyglądać tak:
<br>
<br>
![wynik](./images/table.png)


### Zadanie 2:
Postawić prosty serwis internetowy z wykorzystaniem Flask, bazując na QuickGuide ze strony https://flask.palletsprojects.com/en/3.0.x/quickstart#a-minimal-application

Plik 'index.html':
```html
<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8" />
      <meta http-equiv="X-UA-Compatible" content="IE=edge" />
      <meta name="viewport" content="width=device-width, initial-scale=1.0" />
      <title>Flask Example</title>
   </head>
   <body>
      <p>Enter your favorite game:</p>
      <form action="/game" method="post">
         <input type="text" name="game" required />
         <input type="submit" value="Grade game!" />
      </form>
   </body>
</html>
```

Plik 'game.html':
```html
<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8" />
      <meta http-equiv="X-UA-Compatible" content="IE=edge" />
      <meta name="viewport" content="width=device-width, initial-scale=1.0" />
      <title>{{ game }}</title>
   </head>
   <body>
      <h1>Game {{ game }} is the best!</h1>
   </body>
</html>
```

In [ ]:
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/game', methods=['POST', 'GET'])
def hello():
    if request.method == 'POST':
        game = request.form['game']
        return render_template('game.html', game=game)
    else:
        return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)


Po uruchomieniu programu, w terminalu powinien pojawić się link do strony:
```* Serving Flask app 'main'
 * Debug mode: on
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 612-408-260
 ``````

Strona powinna wyglądać tak:
<br>
![wynik](./images/ind.png)
<br>
A po napisaniu nazwy gry i kliknięciu przycisku, strona powinna wyglądać tak:
<br>
![wynik](./images/gam.png)

### Zadanie 3:
Utworzyć serwis internetowy oferujące API z wybranej dziedziny (obliczenia, obsługa tekstów, przetwarzanie danych) z wykorzystaniem FastAPI bazując na First steps ze strony https://fastapi.tiangolo.com/tutorial/first-steps/

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str
    password: str

In [ ]:
from fastapi import FastAPI
from models import User

app = FastAPI()

users = []

@app.get("/users")
async def get_users():
    return users

@app.get("/users/{user_id}")
async def get_user(user_id: int):
    if 1 <= user_id <= len(users):
        return users[user_id - 1]
    else:
        return {"message": f"User with ID {user_id} doesn't exist"}

@app.post("/users")
async def create_user(user: User):
    users.append(user)
    return {"message": "User created successfully"}

@app.put("/users/{user_id}")
async def update_user(user_id: int, updated_user: User):
    if 1 <= user_id <= len(users):
        users[user_id - 1] = updated_user
        return {"message": f"User with ID {user_id} updated successfully"}
    else:
        return {"message": f"User with ID {user_id} doesn't exist"}

@app.delete("/users/{user_id}")
async def delete_user(user_id: int):
    if 1 <= user_id <= len(users):
        users.pop(user_id - 1)
        return {"message": f"User with ID {user_id} deleted successfully"}
    else:
        return {"message": f"User with ID {user_id} doesn't exist"}


Uruchomamy program w terminalu:
```uvicorn main:app --reload```
<br>
I na stronie 127.0.0.1:8000/docs/ powinno pojawić się API:
<br>
![wynik](./images/fastapi.png)

### Zadanie 4:
Utworzyć prostą aplikacje z GUI i wykorzystaniem biblioteki requests pokazującą wykorzystanie API z zadania 3

In [ ]:
import tkinter as tk
from tkinter import messagebox
import requests

url = 'http://localhost:8000'

class CRUDApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Tkinter CRUD App")
        self.root.geometry("400x400")

        self.get_button = tk.Button(root, text="Get Users", command=self.get_users)
        self.get_button.pack()

        self.label = tk.Label(root, text="User ID:")
        self.label.pack()

        self.user_id_entry = tk.Entry(root)
        self.user_id_entry.pack()

        self.name_label = tk.Label(root, text="Name:")
        self.name_label.pack()

        self.name_entry = tk.Entry(root)
        self.name_entry.pack()

        self.email_label = tk.Label(root, text="Email:")
        self.email_label.pack()

        self.email_entry = tk.Entry(root)
        self.email_entry.pack()

        self.password_label = tk.Label(root, text="Password:")
        self.password_label.pack()

        self.password_entry = tk.Entry(root)
        self.password_entry.pack()

        self.create_button = tk.Button(root, text="Create User", command=self.create_user)
        self.create_button.pack()

        self.update_button = tk.Button(root, text="Update User", command=self.update_user)
        self.update_button.pack()

        self.delete_button = tk.Button(root, text="Delete User", command=self.confirm_delete_user)
        self.delete_button.pack()

    def get_users(self):
        try:
            response = requests.get(f"{url}/users")
            users_data = response.json()
            messagebox.showinfo("Users Details", str(users_data))
        except Exception as e:
            messagebox.showerror("Error", f"Error: {e}")

    def create_user(self):
        try:
            user_id = self.user_id_entry.get()
            name = self.name_entry.get()
            email = self.email_entry.get()
            password = self.password_entry.get()

            response = requests.post(f"{url}/users", json={
                "id": user_id,
                "name": name,
                "email": email,
                "password": password
            })
            messagebox.showinfo("Success", "User created successfully")
        except Exception as e:
            messagebox.showerror("Error", f"Error: {response.json()['message']}")
            messagebox.showerror("Error", f"Error: {e}")

    def update_user(self):
        try:
            response = requests.put(f"{url}/users/{self.user_id_entry.get()}", json={
                "id": self.user_id_entry.get(),
                "name": self.name_entry.get(),
                "email": self.email_entry.get(),
                "password": self.password_entry.get()
            })
            messagebox.showinfo("Success", "User updated successfully")
        except Exception as e:
            messagebox.showerror("Error", f"Error: {response.json()['message']}")
            messagebox.showerror("Error", f"Error: {e}")

    def confirm_delete_user(self):
        confirmation = messagebox.askokcancel("Delete Confirmation", "Are you sure you want to delete this user?")
        if confirmation:
            self.delete_user()

    def delete_user(self):
        try:
            response = requests.delete(f"{url}/users/{self.user_id_entry.get()}")
            messagebox.showinfo("Success", "User deleted successfully")
        except Exception as e:
            messagebox.showerror("Error", f"Error: {response.json()['message']}")
            messagebox.showerror("Error", f"Error: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = CRUDApp(root)
    root.mainloop()


Tworzenie nowego użytkownika:
<br>
![wynik](./images/new_user.png)
<br>
Pobieranie informacji o użytkownikach:
<br>
![wynik](./images/all_users.png)
<br>
Onowienie informacji o użytkowniku:
<br>
![wynik](./images/update_user.png)
<br>
Pobieranie informacji o użytkownikach (widzimy, że użytkownik został zaktualizowany):
<br>
![wynik](./images/all_users_updated.png)
Usunięcie użytkownika:
<br>
![wynik](./images/delete_user.png)
<br>
Pobieranie informacji o użytkownikach (widzimy, że użytkownik został usunięty):
<br>
![wynik](./images/empty_users.png)

## Wnioski:
W trakcie wykonywania ćwiczenia zapoznałem się z zasadami budowania aplikacji sieciowych w języku Python, z wykorzystaniem HTML i REST. Poznałem biblioteki Jinja 2, Flask, FastAPI, requests.

Tworzenie aplikacji z GUI i korzystanie z biblioteki requests do demonstracji obsługi API było niezwykle ciekawe. Nigdy wcześniej nie było mi tak łatwo tworzyć API, jak przy użyciu FastAPI! 👁👁

Więc to było dobrze ćwiczenie i dało mi nowych informacji dla tworzenia biznesu